# Crawling book reviews

This section aims to crawl book reviews for Chinese Xianxia novels on the Wuxia World website.

The essence and basic process of web crawling, please refer to https://rayobyte.com/blog/web-crawling-vs-web-scraping/

In this case, we chose to use Selenium because the Wuxia World website has implemented an anti-crawling mechanism that requires browser simulation to obtain complete data. For installing and documentation, please refer to https://selenium-python.readthedocs.io/index.html

Note: Please make sure to install the necessary libraries such as Selenium and BeautifulSoup before running the code. You can use pip, the Python package installer, to install these libraries. To install them, open your command prompt or terminal and enter the following commands:

pip install selenium;
pip install beautifulsoup4

In [264]:
# Importing all the needed libraries for this project
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

Here, we take ChromeDriver as an example. Please follow the setup instruction https://sites.google.com/chromium.org/driver/getting-started

In [265]:
# Specify the path of the chromedriver and launch the browser
driver = webdriver.Chrome(executable_path='/Users/wanshuo/Desktop/Master/DH_MA_thesis/Dataset/chromedriver_mac_arm64/chromedriver')

# Open the novel page
driver.get('https://www.wuxiaworld.com/novel/sovereign-of-the-three-realms')

<ipython-input-265-db031d3a039e>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='/Users/wanshuo/Desktop/Master/DH_MA_thesis/Dataset/chromedriver_mac_arm64/chromedriver')


In [266]:
# Locate the "View All" button and click on it
view_all_link = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="full-width-tabpanel-0"]/div/div[3]/div[2]/div[2]/div/div[2]/div/span'))
)
view_all_link.click()

In [267]:
# Wait for 10 seconds for the comments to load
time.sleep(10)

In [268]:
# Get the webpage source code and parse the HTML code using BeautifulSoup
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

In [269]:
# Find all the comment divs and store them in a list
data = soup.find_all('div',{'class':'absolute'})

In [270]:
# Find the next page button
next_page = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, 
                                    '/html/body/div[2]/div[3]/div/div/div/div[2]/div[3]/nav/ul/li[9]/button'))
)

In [271]:
# Create an empty list to store all the comments
comments = []

In [272]:
# keep clicking the next page button until it's disabled
while next_page.is_enabled():
    next_page.click()
    time.sleep(10)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    data = soup.find_all('div', class_="absolute top-0 -z-10 line-clamp-1 font-set-r15-h150 text-gray-t1 sm2:font-set-r16-h150")[3:]
    comments.extend(data)
    next_page = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, 
                                    '/html/body/div[2]/div[3]/div/div/div/div[2]/div[3]/nav/ul/li[9]/button'))
    )

In [273]:
# export the comments to a txt file
with open('comments.txt', 'w', encoding='utf-8') as f:
    for comment in comments:
        f.write(comment.get_text(strip=True))# write only the text content of the comment to the file
        f.write('\n\n')# create a blank line between each comment

In [274]:
# close the browser after exporting the comments
driver.quit()